# HW7: Contingency tables

We'll use simple 2x2 contingency tables to explore overrepresented words (using Dunning's log-likelihood a.k.a a G-test), and then use a contingency table of a similar form to dramatize the importance of distinguishing "recall" from "precision" in classification.

We'll start by importing some familiar modules, as well as "display" which will allow us to directly print dataframes. We're also importing a function called ```chi2_contingency``` that we will eventually use to calculate Dunning's log likelihood.

In [1]:
import re, random, math
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import cross_validate
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import cross_val_predict
from scipy.stats import chi2_contingency
from IPython.display import display

## Dunning's log likelihood

Ultimately our goal here is to explore a more principled way of identifying "overrepresented words" than the simple ratio tests we were using earlier in the course.

We'll start by reading in the familiar movie-characters dataset.

In [2]:
dialogpath = Path('../data/movie_dialogue.tsv')

chars = pd.read_csv(dialogpath, sep = '\t')

# let's also randomize the row order
chars = chars.sample(frac = 1)
chars.head()

,mid,cid,cname,mname,gender,wordcount,year,genres,comedy,thriller,drama,romance,lines
2355,m536,u7921,GENERAL SCHMUCK,dr. strangelove or: how i learned to stop worr...,M,828,1964,"['comedy', 'drama']",True,False,True,False,"That's right, fella. / See! See, I told you. ..."
1145,m303,u4593,JONAS,conspiracy theory,m,555,1997,"['action', 'crime', 'mystery', 'romance', 'thr...",False,True,False,True,"Liza Sutton is dead. / You shouldn't watch, Je..."
652,m214,u3238,OVERLORD,the time machine,M,421,2002,"['sci-fi', 'adventure', 'action']",False,False,False,False,And why not? / But you've earned a reward for ...
1445,m363,u5452,LILLIAN,game 6,f,165,2005,"['comedy', 'drama', 'sport']",True,False,True,False,I'm glad we're having this talk. / I never tho...
96,m113,u1725,YNYR,krull,m,1173,1983,"['fantasy', 'action', 'adventure', 'sci-fi']",False,False,False,False,I can. Because I choose it. / It is my fate to...


To frame a useful question: How does movie dialogue change across time? What counts as "old-fashioned" or "classic" language in the movies?

Let's distinguish two groups of films, not by using the *median* year for characters--which we've already calculated as somewhere in the 1990s--but the actual midpoint of the timeline.

In [5]:
(max(chars['year']) + min(chars['year'])) / 2

1968.5

Let's construct two lists of character IDs based on release date of the movie.

In [6]:
classics = chars.loc[chars['year'] < 1968.5, 'cid']
recents = chars.loc[chars['year'] > 1968.5, 'cid']

print('We have ' + str(len(classics)) + ' classic characters and')
print(str(len(recents)) + ' more recent characters.')

We have 358 classic characters and
2611 more recent characters.


What words are used especially in older movies?

We've explored several casual ways of answering that question, but a more rigorous way to pose it is Dunning's log-likelihood. You may remember that early in the course we read [Ben Schmidt's intuitive explanation of why Dunning's is better than a simple ratio, or simple subtraction.](http://sappingattention.blogspot.com/2011/10/comparing-corpuses-by-word-use.html) But we didn't learn how to actually calculate it in Python.

In form, Dunning's test works very much like a chi-squared test for the difference of two proportions. In both cases you begin by constructing a simple *contingency table.*

But before we can do that we need to

#### generate the term-doc matrix for movie characters

In [7]:
if 'cid' in chars.columns:         
    character_ids = chars['cid']
    chars = chars.set_index('cid')   # If we haven't made char-id the index yet, let's do it.
else:
    character_ids = chars.index.tolist()

vectorizer = CountVectorizer(min_df = 0.01)  # this limits the table to words present in
                                             # at least 1% of characters

chars.reset_index(inplace = True)
    
sparse_counts = vectorizer.fit_transform(chars['lines']) # the vectorizer produces something
                                                               # called a 'sparse matrix'; we need to
                                                               # unpack it
        
wordcounts = pd.DataFrame(sparse_counts.toarray(), index = character_ids, 
                            columns = vectorizer.get_feature_names())
wordcounts.shape

(2969, 3359)

In [8]:
wordcounts.head()

,000,10,100,11,12,15,18,20,24,25,...,younger,your,yours,yourself,yourselves,youth,yup,zero,zone,zoo
cid,,,,,,,,,,,,,,,,,,,,,
u7921,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
u4593,0,0,0,0,0,0,0,0,0,0,...,0,3,0,1,0,0,0,0,0,0
u3238,0,0,0,0,0,0,0,0,0,0,...,0,3,1,0,0,0,0,0,0,0
u5452,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
u1725,0,0,0,0,0,0,0,0,0,0,...,0,6,1,1,0,0,0,0,0,0


Let's now see whether the word *swell* is overrepresented in old movies.

#### building a contingency table

Let's start by counting occurrences of "swell."

In [9]:
classicswell = wordcounts.loc[classics, 'swell'].sum()
recentswell = wordcounts.loc[recents, 'swell'].sum()

To do the contingency table we also need to know how many words that aren't "swell" appear in both categories.

In [10]:
allclassicwords = wordcounts.loc[classics, ].sum().sum()
allrecentwords = wordcounts.loc[recents, ].sum().sum()

classicnotswell = allclassicwords - classicswell
recentnotswell = allrecentwords - recentswell

Now we can build the table.

In [11]:
c_table = pd.DataFrame({'swell': [classicswell, recentswell], 
                        'not swell': [classicnotswell, recentnotswell]},
                      index = ['classics', 'recent'])

display(c_table)

,swell,not swell
classics,41,287224
recent,51,1893313


It looks like *swell* is proportionally more common in old movies, even though there are more occurrences in recent movies. But how can we know whether this is a significant difference?

The Dunning's log-likelihood formula is

$$G = 2\sum_{i=1}^n O_i \ln(\frac{O_i}{E_i})$$

where $O_i$ is the observed frequency in each of the four cells of the table, and $E_i$ is the expected frequency in each cell under the null hypotheses of even distribution across classes. We calculate *G* by adding up observed times the log of (observed/expected) in each cell.

You can start to construct the null contingency table with expected values by calculating the **marginal** frequencies for the observed table, which basically means summing both the rows and the columns.

#### marginal frequencies

In [12]:
rowsums = c_table.sum(axis = 'columns')
colsums = c_table.sum(axis = 'rows')
colsums = colsums.rename('totals')

In [13]:
e_table = c_table.append(colsums)
e_table['all words'] = rowsums
display(e_table)

,swell,not swell,all words
classics,41,287224,287265.0
recent,51,1893313,1893364.0
totals,92,2180537,NaN


To construct the table of expected counts we need to express one of these margins as a probability. In other words, how likely is it that a given word comes from a classic movie, or a recent movie?

#### marginal probabilities

In [14]:
e_table['marginal probs'] = rowsums / np.sum(rowsums)
display(e_table)

,swell,not swell,all words,marginal probs
classics,41,287224,287265.0,0.131735
recent,51,1893313,1893364.0,0.868265
totals,92,2180537,NaN,NaN


Now by multiplying both of the "totals" (counts of words in swell / notswell classes) by these marginal probabilities, we can construct a table of the expected distribution.

In [15]:
expected = colsums.apply(lambda r: r * e_table['marginal probs'][0:2])
expected = expected.transpose()
display(expected)

,swell,not swell
classics,12.119613,2.872529e+05
recent,79.880387,1.893284e+06


Now we can apply the formula for G to calculate it. Because I really love putting LaTeX math notation in markdown, let's have that formula again!

$$G = 2\sum_{i=1}^n O_i \ln(\frac{O_i}{E_i})$$

In [16]:
G = 0

for film_cat in ['classics', 'recent']:
    for word_cat in ['swell', 'not swell']:
        observe = c_table.loc[film_cat, word_cat]
        expect = expected.loc[film_cat, word_cat]
        G += 2 * observe * math.log(observe/expect)
G   

54.17271712337733

Which is a step-by-step explanation of what happens when we use this sklearn function:

In [47]:
G, p, dof, ex = chi2_contingency(c_table, lambda_ = 'log-likelihood')
print(G, p)
print(ex)

52.51615912666403 4.26722744688518e-13
[[1.21196132e+01 2.87252880e+05]
 [7.98803868e+01 1.89328412e+06]]


The G value is very slightly different; I'm not sure but I *think* that's simply because the sklearn implementation is approximating. The sklearn version also calculates a p-value for us, so we know this divergence would be very unlikely to happen in a world where "swell" was actually equally common in both classes.

If we want to sort words by overrepresentation in one category or the other, we'll need to add a sign, because the log-likelihood test itself only tells us how *far* this distribution is from the null model — not which direction.

In [17]:
def signed_dunnings(wordcounts, catA, catB):
    Asum = wordcounts.loc[catA, ].sum().sum()
    Bsum = wordcounts.loc[catB, ].sum().sum()
    
    tuples = []
    for word in wordcounts.columns:
        Acount = wordcounts.loc[catA, word].sum()
        Bcount = wordcounts.loc[catB, word].sum()
        
        if Acount < 6 or Bcount < 6:   # it's hard to be confident about the significance
            continue                   # of very small numbers
            
        table = np.array([[Acount, Asum - Acount], [Bcount , Bsum - Bcount]])
        G, p, dof, ex = chi2_contingency(table, lambda_ = 'log-likelihood')
        
        if Acount / Asum < Bcount / Bsum:
            G = 0 - G
        
        tuples.append((G, word))

    tuples.sort()
    return tuples

In [18]:
earlywords = signed_dunnings(wordcounts, classics, recents)

#### in-class

Explore that list of tuples to see what vocabulary distinguishes early movies. 

### QUESTION 1

Use the same technique to find vocabulary that characterizes comedies. Just the high end will do: 25 words *over*represented in comedies.

## Recall and precision in classification of imbalanced classes

I'm going to set up a logistic-regression model of early movies, using code borrowed from the March 15th "Word Vectors" lab.

Then I'll ask you to calculate recall and precision using a contingency table like the one above.

In [21]:
rowsums = wordcounts.sum(axis = 'columns')
frequencies = wordcounts.divide(rowsums, axis = 'rows')
frequencies.head()

,000,10,100,11,12,15,18,20,24,25,...,younger,your,yours,yourself,yourselves,youth,yup,zero,zone,zoo
cid,,,,,,,,,,,,,,,,,,,,,
u7921,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.002759,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
u4593,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.006424,0.000000,0.002141,0.0,0.0,0.0,0.0,0.0,0.0
u3238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.008310,0.002770,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
u5452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.007299,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
u1725,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.005709,0.000951,0.000951,0.0,0.0,0.0,0.0,0.0,0.0


You'll recall that logistic regression works best when features are "scaled": each column is transformed by subtracting the column mean and dividing by column standard deviation. 

In [22]:
scaler = StandardScaler()
X_normed = scaler.fit_transform(frequencies)
y = chars['year'] < 1968.5

Because our classes are pretty imbalanced (there are more recent movies than classic films), we'll tune this model to maximize the f1 score, which reflects both recall and precision.

In order to do that, we make a couple of crucial decisions below. The class_weight parameter sets the *importance* of each example in the regression to be *inversely* proportional to class size, so it costs more to be wrong about rare classes.

We also set a scoring parameter that reports the f1 score rather than simple accuracy.

In [ ]:
for c_param in [10, 1, .1, .02, .01, .005, .001, .0001, .00001, .000001]:
    balanced_logreg = LogisticRegression(C = c_param, max_iter = 1000, class_weight = 'balanced') # note the class weight
    grouper = GroupKFold(n_splits = 10)
    cv_results = cross_validate(balanced_logreg, X_normed, y, 
                                groups = chars['mid'], cv = grouper, scoring = 'f1')  # note the scoring
    mean_score = np.mean(cv_results['test_score'])
    print(c_param, mean_score)

The F1 score, reported above, can be expressed mathematically in several ways:

$$2 \cdot \frac{\text{precision} \cdot \text{recall}}{\text{precision} + \text{recall}}$$

or alternately

$$\frac{\text{tp}}{\text{tp} + \frac{1}{2} (\text{fp} + \text{fn})}$$

Why do we need to use this complex measure?

To understand intuitively why we need it, let's look at the number of true positives (correctly identified classic movies) when we tune the model in different ways. 

We can start by getting the predictions from the best model above, and comparing them to the real class labels.

In [80]:
balanced_logreg = LogisticRegression(C = .0001, max_iter = 1000, class_weight = 'balanced')

predictions = cross_val_predict(balanced_logreg, X_normed, y, groups = chars['mid'], cv = grouper, method='predict')
predictions[0:10]

array([False, False, False, False,  True, False, False, False, False,
        True])

In [81]:
y[0:10]

0    False
1    False
2    False
3    False
4     True
5    False
6    False
7    False
8    False
9     True
Name: year, dtype: bool

### QUESTION 2

Using those two sequences, produce a contingency table of the form below:

In [84]:
dummy_c_table = pd.DataFrame({'predicted classic': ['true positives', 'false positives'], 
                        'predicted not classic': ['false negatives', 'true negatives']},
                      index = ['actually classic', 'actually not classic'])

display(dummy_c_table)

,predicted classic,predicted not classic
actually classic,true positives,false negatives
actually not classic,false positives,true negatives


Note that the code we used to create a table for Dunning's is not really very useful here, because your data is in a completely different format (two lists). The only similarity is that the result can in both cases take the form of a 2x2 contingency table.

Then calculate recall, precision, and f1 score for this table. You can look up formulas for each of these, but for easy reference.

$$\text{precision} = \frac{tp}{tp + fp}$$

$$\text{recall} = \frac{tp}{tp + fn}$$

The formula for f1 is provided a few cells above. Your final f1 score should come out fairly close to the f1 score calculated above when we optimized the c parameter.

### QUESTION 3

Try tuning a model without the special parameters we used above to handle imbalanced classes; that is,

    class_weight = 'balanced')
    scoring = 'f1')

Your new model will treat both classes equally, and will try to maximize accuracy, which is the default scoring criterion. Get the predictions from this model, and produce another contingency table. How many true positives does the new model give you?